In [ ]:
#Load Data
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px
from Quantapp.Algorithm   import Algorithm
from Quantapp.Computation import Computation
from Quantapp.Portfolio   import Portfolio
from Quantapp.Plot        import Plot
from Quantapp.DataManager import DataManager
from Quantapp.Universe    import Universe

csv_file_paths = [
    'csv_files/S&P 500.csv'
]

dm   = DataManager()
universe  = Universe(file_paths = csv_file_paths)
pm   = Portfolio(universe = universe)
algorithms = Algorithm()
comp= Computation()

plt.rcParams["figure.figsize"] = (10, 10)

period= '2y'
interval='1d'
n = 200
market_cap_threshold=1000000000
Sector = 'Information Technology'
Industry ='Semiconductors & Semiconductor Equipment'
Sub_Industry= 'Semiconductors'

stocks    = universe.retrieve_assets(Sector=Sector, Industry=Industry)
tickers = stocks['Symbol']

for ticker in tickers:
    pm.add(ticker)
pm.load()
tickers = pm.retrieve_ticker_data()



In [ ]:
#Industry Rate of Change
industries = pd.Series(universe.retrieve_unique_asset_classes(Sector=Sector))
industry_indices = pd.DataFrame()

for industry in industries:
   industry_indices[industry] = universe.create_market_index(Industry=industry)

industry_indices
formula = algorithms.rate_of_change

industry_index_roc_50 = comp.rolling_multi(formula,50,industry_indices)
industry_index_roc_200= comp.rolling_multi(formula,200,industry_indices)

px.line(industry_index_roc_50,title='Rate of Change (50 day)').show()
px.line(industry_index_roc_200,title='Rate of Change (200 day)').show()


In [ ]:
#Sub-Industrys Rate of Change 
sub_industries = pd.Series(universe.retrieve_unique_asset_classes(Industry=Industry))
sub_industry_indices = pd.DataFrame()
for sub_industry in sub_industries:
    sub_industry_indices[sub_industry] = universe.create_market_index(SubIndustry=sub_industry)

formula = algorithms.rate_of_change
sub_industry_index_roc_50 = comp.rolling_multi(formula,50,sub_industry_indices)
sub_industry_index_roc_200= comp.rolling_multi(formula,200,sub_industry_indices)

px.line(sub_industry_index_roc_50,title='Rate of Change (50 day)').show()
px.line(sub_industry_index_roc_200,title='Rate of Change (200 day)').show()

In [ ]:
#Industry vs Sub Industry 50D & 200D ROC 
df = pd.DataFrame()
df[Industry] = industry_index_roc_50[Industry]
df[Sub_Industry] = sub_industry_index_roc_50[Sub_Industry]
px.line(df,title='Rate of Change (50 day)').show()

df = pd.DataFrame()
df[Industry] = industry_index_roc_200[Industry]
df[Sub_Industry] = sub_industry_index_roc_200[Sub_Industry]
px.line(df,title='Rate of Change (200 day)').show()

In [ ]:
market_index = universe.create_market_index(Sector=Sector,Industry=Industry,SubIndustry=Sub_Industry)
assets       = universe.retrieve_assets(Sector=Sector,Industry=Industry,SubIndustry=Sub_Industry).sort_values(by=['Market Cap'], ascending=False)

tickers      = {}
for stock in assets['Symbol']:
    tickers[stock] = 'long'

print(tickers)
pm.add_tickers(tickers)
pm.load(period=period,interval=interval)

In [ ]:
assets

In [ ]:
market_caps = assets[['Symbol', 'Market Cap']].sort_values(by=['Market Cap'], ascending=False)
fig = px.bar(market_caps, x='Symbol',y='Market Cap', title=" Market Caps ")
fig.show()
market_cap_vs_correlation = pd.DataFrame(
    {
        'tickers'   : assets['Symbol'].to_list(),
        'correlation' : pm.benchmark_correlation('XLF',period=period,interval=interval,window=252*2).iloc[-1,:],
        'market_caps'  : market_caps['Market Cap'].to_list()
    }
)
fig2 = px.line(market_cap_vs_correlation,x='tickers',y='correlation',text='tickers')
fig2.add_hline(0)
fig2.add_hline(.5)
fig2.show()

In [ ]:
df = computations.create_correlation_heatmap(industry_indices)
fig_ = px.imshow(df,text_auto=True,width=600,height=600,title='fgfgk')
fig_.show()

df2 = computations.create_correlation_heatmap(sub_industry_indices)
fig2_ = px.imshow(df2,text_auto=True,width=600,height=600,title='fgfgk')
fig2_.show()

df3= computations.create_correlation_heatmap(pm.retrieve_ticker_data())
fig3_ = px.imshow(df3,text_auto=True,width=600,height=600,title='fgfgk')
fig3_.show()


In [ ]:
statistics = Statistics()
positive_trading_days_21 = statistics.n_positive_days(ticker='XLV',number_of_days=21)
positive_trading_days_50 = statistics.n_positive_days(ticker='XLV',number_of_days=50)
positive_trading_days_200 = statistics.n_positive_days(ticker='XLV',number_of_days=200)

In [ ]:
positive_trading_days_21[-252*3:].plot()
positive_trading_days_50[-252*3:].plot()
positive_trading_days_200[-252*3:].plot()